#Importing Necessary Libraries

In [50]:
# -------------------------------
# Imports and library setup
# -------------------------------
import re
import os
import shutil
import kagglehub
import numpy as np
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

In [51]:
# -------------------------------
# Download dataset using KaggleHub
# -------------------------------

In [52]:
# Download latest version
path = kagglehub.dataset_download("ashishpandey2062/next-word-predictor-text-generator-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'next-word-predictor-text-generator-dataset' dataset.
Path to dataset files: /kaggle/input/next-word-predictor-text-generator-dataset


In [53]:
# -------------------------------
# List input files and set up local directory
# -------------------------------
# List Kaggle input files and prepare destination directory for local copy

In [54]:
os.listdir("/kaggle/input")
Source_Dir="/kaggle/input/next-word-predictor-text-generator-dataset"
Dest_Dir="/content/data"
os.makedirs(Dest_Dir,exist_ok=True)

In [55]:
# -------------------------------
# Copy dataset to local working directory
# -------------------------------
# Copy dataset folder to /content/data for processing

In [56]:
shutil.copytree(
    Source_Dir,
    os.path.join(Dest_Dir, os.path.basename(Source_Dir)),
    dirs_exist_ok=True
)

'/content/data/next-word-predictor-text-generator-dataset'

In [57]:
# -------------------------------
# Load text file from dataset
# -------------------------------
# Read the main text file into a string variable

In [58]:
data_path = "/content/data/next-word-predictor-text-generator-dataset/next_word_predictor.txt"

with open(data_path, "r", encoding="utf-8", errors="ignore") as f:
    text_data = f.read()


In [59]:
# Quick sanity check of dataset
print(type(text_data))        # should be <class 'str'>
print(len(text_data))         # should be > 10000 (ideally)
print(text_data[:200])        # readable text


<class 'str'>
167445
The sun was shining brightly in the clear blue sky, and a gentle breeze rustled the leaves of the tall trees. People were out enjoying the beautiful weather, some sitting in the park, others taking a 


In [60]:
# -------------------------------
# Preprocess the text
# -------------------------------
# Clean text by lowercasing, removing non-alphanumeric chars, and stripping extra spaces


In [61]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

corpus = preprocess_text(text_data)

In [62]:
# -------------------------------
# Tokenize the text
# -------------------------------
# Fit a Keras tokenizer on the cleaned corpus and calculate vocabulary size

In [63]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts([corpus])

total_words = len(tokenizer.word_index) + 1
print("Vocabulary size:", total_words)

tokens = tokenizer.texts_to_sequences([corpus])[0]


Vocabulary size: 5012


In [64]:
# -------------------------------
# Create sequences for training
# -------------------------------
# Generate fixed-length sequences of tokens for input and next-word prediction

In [65]:
SEQUENCE_LENGTH = 30

input_sequences = []

for i in range(SEQUENCE_LENGTH, len(tokens)):
    seq = tokens[i-SEQUENCE_LENGTH:i+1]
    input_sequences.append(seq)

input_sequences = np.array(input_sequences)

X_train = input_sequences[:, :-1]
y_train = input_sequences[:, -1]

print("X shape:", X_train.shape)
print("y shape (before one-hot):", y_train.shape)


X shape: (27598, 30)
y shape (before one-hot): (27598,)


In [66]:
# -------------------------------
# One-hot encode labels
# -------------------------------
# Convert y labels to categorical one-hot vectors

In [67]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=total_words)

print("X shape:", X_train.shape)
print("y shape:", y_train.shape)


X shape: (27598, 30)
y shape: (27598, 5012)


In [68]:
# -------------------------------
# Build LSTM model
# -------------------------------
# Define sequential LSTM model with embedding, dropout, and dense layers

In [69]:


SEQUENCE_LENGTH = 30

model = Sequential([
    Input(shape=(SEQUENCE_LENGTH,)),
    Embedding(input_dim=total_words, output_dim=128),
    LSTM(256, return_sequences=True),
    Dropout(0.2),
    LSTM(256),
    Dense(256, activation="relu"),
    Dense(total_words, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 30, 128)        │       641,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 30, 256)        │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 30, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 256)            │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 5012)           │     1,288,084 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,914,964 (11.12 MB)

 Trainable params: 2,914,964 (11.12 MB)

 Non-trainable params: 0 (0.00 B)

In [70]:
# -------------------------------
# Define callbacks for training
# -------------------------------
# Commit: Save best model and implement early stopping to prevent overfitting

In [71]:
checkpoint = ModelCheckpoint(
    filepath="best_next_word_model.h5",
    monitor="loss",
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

early_stop = EarlyStopping(
    monitor="loss",
    patience=5,
    restore_best_weights=True,
    verbose=1
)


In [43]:
# -------------------------------
# Train the model
# -------------------------------
# Commit: Fit model on training sequences with checkpoints and early stopping

In [26]:
history = model.fit(
    X_train,
    y_train,
    epochs=70,
    batch_size=64,
    callbacks=[checkpoint, early_stop]
)


Epoch 1/70
431/432 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2368 - loss: 3.4327
Epoch 1: loss improved from 3.64494 to 3.51193, saving model to best_next_word_model.h5


432/432 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.2367 - loss: 3.4330
Epoch 2/70
430/432 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2618 - loss: 3.2917
Epoch 2: loss improved from 3.51193 to 3.36873, saving model to best_next_word_model.h5


432/432 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.2617 - loss: 3.2922
Epoch 3/70
432/432 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2804 - loss: 3.1481
Epoch 3: loss improved from 3.36873 to 3.23332, saving model to best_next_word_model.h5


432/432 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.2803 - loss: 3.1483
Epoch 4/70
432/432 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3015 - loss: 3.0158
Epoch 4: loss improved from 3.23332 to 3.10391, saving model to best_next_word_model.h5


432/432 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.3015 - loss: 3.0160
Epoch 5/70
431/432 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3298 - loss: 2.8925
Epoch 5: loss improved from 3.10391 to 2.98446, saving model to best_next_word_model.h5


...

432/432 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9381 - loss: 0.2309
Epoch 63/70
432/432 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9506 - loss: 0.1949
Epoch 63: loss improved from 0.24564 to 0.21694, saving model to best_next_word_model.h5


432/432 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9506 - loss: 0.1949
Epoch 64/70
431/432 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9442 - loss: 0.2141
Epoch 64: loss did not improve from 0.21694
432/432 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9442 - loss: 0.2142
Epoch 65/70
431/432 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9517 - loss: 0.1927
Epoch 65: loss improved from 0.21694 to 0.20696, saving model to best_next_word_model.h5


432/432 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9517 - loss: 0.1927
Epoch 66/70
431/432 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9515 - loss: 0.1849
Epoch 66: loss improved from 0.20696 to 0.19456, saving model to best_next_word_model.h5


432/432 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9515 - loss: 0.1850
Epoch 67/70
429/432 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9563 - loss: 0.1752
Epoch 67: loss improved from 0.19456 to 0.18462, saving model to best_next_word_model.h5


432/432 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9562 - loss: 0.1753
Epoch 68/70
430/432 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9537 - loss: 0.1753
Epoch 68: loss did not improve from 0.18462
432/432 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9536 - loss: 0.1755
Epoch 69/70
431/432 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9508 - loss: 0.1817
Epoch 69: loss did not improve from 0.18462
432/432 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9508 - loss: 0.1818
Epoch 70/70
429/432 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9471 - loss: 0.1965
Epoch 70: loss did not improve from 0.18462
432/432 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9471 - loss: 0.1966
Restoring model weights from the end of the best epoch: 67.


In [72]:
# -------------------------------
# Load best saved model
# -------------------------------
# Commit: Load the best performing model from checkpoint

In [73]:
model = load_model("best_next_word_model.h5")

In [74]:
# -------------------------------
# Define next-word prediction function
# -------------------------------
# Commit: Generate multiple next words in a loop using temperature-based sampling

In [75]:
def predict_next_words(model, tokenizer, seed_text, num_words=10, sequence_length=30, temperature=1.0):
    """
    Predicts next words in a loop using a trained model.

    Args:
        model: Trained Keras model
        tokenizer: Fitted Keras Tokenizer
        seed_text: Input text to start predictions
        num_words: Number of words to generate
        sequence_length: Length of input sequences used during training
        temperature: Sampling temperature (higher = more random)

    Returns:
        Generated text including seed_text and predicted words
    """
    output_text = seed_text

    for _ in range(num_words):
        # Preprocess seed_text
        text = output_text.lower()
        text = text.strip()

        # Convert to sequence
        sequence = tokenizer.texts_to_sequences([text])[0]

        # Pad sequence
        sequence = pad_sequences([sequence], maxlen=sequence_length, padding='pre')

        # Predict probabilities
        preds = model.predict(sequence, verbose=0)[0]

        # Apply temperature for more creativity
        preds = np.log(preds + 1e-8) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)

        # Sample the next word index
        next_index = np.random.choice(len(preds), p=preds)

        # Find corresponding word
        for word, index in tokenizer.word_index.items():
            if index == next_index:
                next_word = word
                break

        # Append next word
        output_text += ' ' + next_word

    return output_text


In [76]:
# -------------------------------
# Generate text from user input
# -------------------------------
# Prompt user for seed text and generate 25 predicted words

In [77]:
seed_text = input("Enter your text:")
generated_text = predict_next_words(
    model=model,
    tokenizer=tokenizer,
    seed_text=seed_text,
    num_words=25,
    sequence_length=SEQUENCE_LENGTH,
    temperature=0.8   # Lower = more predictable, Higher = more creative
)

print(generated_text)


Enter your text:The sun was shining brightly
The sun was shining brightly wait ross youre going to joey right maybe dont do that alright now they want you to stay with rachel and me how ive should
